In [16]:
from cgi import test
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn import preprocessing

df = pd.read_csv('./data/n_movies.csv', thousands=',')
cols_to_drop = ["year", "certificate",
                "duration", "genre", "stars", "description"]
# first five rows of dataframe after removing columns
df = df.drop(columns=cols_to_drop, axis=1)
df = df.dropna()
df["votes"] = df["votes"].astype(int)
df["rating"] = df["rating"].astype(float)
df["title"] = df["title"].astype(str)
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8784 entries, 0 to 9956
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   8784 non-null   object 
 1   rating  8784 non-null   float64
 2   votes   8784 non-null   int32  
dtypes: float64(1), int32(1), object(1)
memory usage: 240.2+ KB


In [17]:
df.corr(numeric_only = True)

,rating,votes
rating,1.000000,0.142622
votes,0.142622,1.000000


In [18]:
X= df['votes'].values.reshape(-1,1)
Y= df['rating'].values.reshape(-1,1)
#Y = lab.fit_transform(Y)
print(Y)
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.4,random_state=100)
logreg= LogisticRegression()
logreg.fit(X_train,np.ravel(y_train,order='C'))
y_pred=logreg.predict(X_test)
print (X_test) #test dataset
print (y_pred) #predicted values

[[8.5]
 [8.7]
 [8.9]
 ...
 [8.7]
 [8.4]
 [5.9]]


ValueError: Unknown label type: 'continuous'